In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect(':memory:')

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [3]:
data = pd.read_csv('boston.csv')
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [4]:
data.to_sql('boston', con, index=False, if_exists='replace')

506

In [5]:
pd.read_sql(
    '''
    SELECT * FROM boston
    limit 5
    ''',
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


### Все задания нужно выполнить только с помощью sql

1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)

In [16]:
pd.read_sql(
    '''
    SELECT 
    'CRIM' AS column_name, 
    SUM(CRIM = 0) AS empty_values
FROM boston
UNION ALL
SELECT 
    'ZN', 
    SUM(ZN = 0) AS empty_values
FROM boston
UNION ALL
SELECT 
    'INDUS', 
    SUM(INDUS = 0) AS empty_values
FROM boston
UNION ALL
SELECT 
    'CHAS', 
    SUM(CHAS = 0) AS empty_values
FROM boston
UNION ALL
SELECT 
    'NOX', 
    SUM(NOX = 0) AS empty_values
FROM boston;
    ''',
    con,
)

,column_name,empty_values
0,CRIM,0
1,ZN,372
2,INDUS,0
3,CHAS,471
4,NOX,0


2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)

In [7]:
pd.read_sql(
    '''
    SELECT 
    'CRIM' as column_name, 
    COUNT(DISTINCT CRIM) as unique_values
FROM boston
UNION ALL
SELECT 
    'ZN', 
    COUNT(DISTINCT ZN)
FROM boston
UNION ALL
SELECT 
    'INDUS', 
    COUNT(DISTINCT INDUS)
FROM boston
UNION ALL
SELECT 
    'CHAS', 
    COUNT(DISTINCT CHAS)
FROM boston
UNION ALL
SELECT 
    'NOX', 
    COUNT(DISTINCT NOX)
FROM boston
    ''',
    con,
)

,column_name,unique_values
0,CRIM,504
1,ZN,26
2,INDUS,76
3,CHAS,2
4,NOX,81


3. Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.
Напишите какой вывод можно сделать по данным в этих колонках

In [22]:
pd.read_sql(
    '''
    WITH medians AS (
    SELECT 
        'CRIM' AS column_name, 
        MIN(CRIM) AS min_value, 
        (SELECT CRIM FROM boston ORDER BY CRIM LIMIT 1 OFFSET (SELECT COUNT(*) FROM boston WHERE CRIM IS NOT NULL) / 2) AS median_value
    FROM boston
    UNION ALL
    SELECT 
        'ZN', 
        MIN(ZN), 
        (SELECT ZN FROM boston ORDER BY ZN LIMIT 1 OFFSET (SELECT COUNT(*) FROM boston WHERE ZN IS NOT NULL) / 2) 
    FROM boston
    UNION ALL
    SELECT 
        'INDUS', 
        MIN(INDUS), 
        (SELECT INDUS FROM boston ORDER BY INDUS LIMIT 1 OFFSET (SELECT COUNT(*) FROM boston WHERE INDUS IS NOT NULL) / 2) 
    FROM boston
    UNION ALL
    SELECT 
        'CHAS', 
        MIN(CHAS), 
        (SELECT CHAS FROM boston ORDER BY CHAS LIMIT 1 OFFSET (SELECT COUNT(*) FROM boston WHERE CHAS IS NOT NULL) / 2) 
    FROM boston
    UNION ALL
    SELECT 
        'NOX', 
        MIN(NOX), 
        (SELECT NOX FROM boston ORDER BY NOX LIMIT 1 OFFSET (SELECT COUNT(*) FROM boston WHERE NOX IS NOT NULL) / 2) 
    FROM boston
)
SELECT 
    column_name
FROM medians
WHERE min_value = median_value;
    ''',
    con,
)


,column_name
0,ZN
1,CHAS


Выводы:
Колонка ZN, представляет собой долю земель под жилую застройку, зонированную под участки площадью более 25 000 кв. футов, имеет медианное значение, равное нулю. Это может свидетельствовать о том, что большинство земель не зонированы под большие участки.
Колонка CHAS, указывает на наличие границы с рекой, также имеет медианное значение, равное нулю. Это означает, что большинство домов не граничат с рекой.

4. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг) среди всех значений LSTAT. Напишите какой вывод можно сделать по этим данным.

In [10]:

pd.read_sql(
    '''
    WITH ranks AS (
    SELECT 
        LSTAT, 
        MEDV, 
        RANK() OVER (ORDER BY LSTAT) as rank
    FROM boston
)
SELECT 
    LSTAT, 
    MEDV, 
    rank
FROM ranks
WHERE MEDV = (SELECT MAX(MEDV) FROM boston)
ORDER BY rank
    ''',
    con,
)

,LSTAT,MEDV,rank
0,1.73,50.0,1
1,1.92,50.0,2
2,2.88,50.0,6
3,2.96,50.0,8
4,2.97,50.0,9
5,3.16,50.0,15
6,3.26,50.0,17
7,3.32,50.0,18
8,3.70,50.0,26
9,3.73,50.0,27


Выводы:
В домах с самой высокой стоимостью (по колонке MEDV) наблюдается более низкий процент населения с низким социальным статусом (LSTAT). Это указывает на то, что более дорогие дома находятся в районах с более высоким социальным статусом населения.

5. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [11]:

pd.read_sql(
    '''
    SELECT 
    CHAS, 
    AVG(MEDV) as avg_cost
FROM boston
GROUP BY CHAS
    ''',
    con,
)

,CHAS,avg_cost
0,0.0,22.093843
1,1.0,28.440000


6. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.

In [12]:

pd.read_sql(
    '''
    SELECT 
    column_name
FROM (
    SELECT 
        'CRIM' as column_name, 
        AVG(CRIM) FILTER (WHERE CHAS = 1) as river_avg, 
        AVG(CRIM) FILTER (WHERE CHAS = 0) as no_river_avg
    FROM boston
    UNION ALL
    SELECT 
        'ZN', 
        AVG(ZN) FILTER (WHERE CHAS = 1), 
        AVG(ZN) FILTER (WHERE CHAS = 0)
    FROM boston
    UNION ALL
    SELECT 
        'INDUS', 
        AVG(INDUS) FILTER (WHERE CHAS = 1), 
        AVG(INDUS) FILTER (WHERE CHAS = 0)
    FROM boston
    UNION ALL
    SELECT 
        'CHAS', 
        AVG(CHAS) FILTER (WHERE CHAS = 1), 
        AVG(CHAS) FILTER (WHERE CHAS = 0)
    FROM boston
    UNION ALL
    SELECT 
        'NOX', 
        AVG(NOX) FILTER (WHERE CHAS = 1), 
        AVG(NOX) FILTER (WHERE CHAS = 0)
    FROM boston
)
WHERE river_avg > no_river_avg
    ''',
    con,
)

,column_name
0,INDUS
1,CHAS
2,NOX


Выводы:
факторы, такие как уровень преступности (CRIM), доля зонированной земли (ZN), доля промышленной застройки (INDUS), концентрация оксидов азота (NOX), могут быть выше в районах, которые граничат с рекой. Это может указывать на определенные экологические или экономические особенности таких районов.

7. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [13]:

pd.read_sql(
    '''
    WITH percentiles AS (
    SELECT 
        NTILE(10) OVER (ORDER BY INDUS) as percentile, 
        INDUS, 
        NOX
    FROM boston
)
SELECT 
    percentile*10 as percentile, 
    AVG(INDUS) as avg_indus, 
    AVG(NOX) as avg_nox
FROM percentiles
GROUP BY percentile
    ''',
    con,
)

,percentile,avg_indus,avg_nox
0,10,2.016078,0.438378
1,20,3.645294,0.494129
2,30,5.246275,0.448235
3,40,6.471765,0.470839
4,50,8.256078,0.531510
5,60,10.749216,0.515176
6,70,17.245800,0.649660
7,80,18.100000,0.675520
8,90,18.307200,0.676480
9,100,21.938800,0.655620


Взаимосвязь между долей промышленной застройки и концентрацией оксидов азота, вероятно, положительная: с увеличением доли промышленной застройки растет и концентрация оксидов азота. Это типично для районов с высокой индустриализацией, где промышленные выбросы влияют на уровень загрязнения воздуха.
